<a href="https://colab.research.google.com/github/alexisnpavlik/Binance-Futures-Calculator/blob/main/SONIC_Experiment_Alexis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set Up

In [1]:
# Prepare the environment
%%capture
from google.colab import drive, auth
drive.mount('/content/drive')
%cd /content/drive/MyDrive/python-library
!pip install .
!sudo apt-get install python-pymysql
!pip install colab-env
!pip install Hashids
import colab_env
%cd /content
import nltk
nltk.download('stopwords')
import gallifrey
import pandas as pd
import numpy as np
import time
from hashids import Hashids

#Set up for Hash calculation
hasher = Hashids("jS5qAlR0kHbAeqeuVowPBnyotKuTvvlGzIzkEtz79XQGB7BJHb", 
                 8, 
                 "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890")

# Authenticate
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())

# create, and save df
from gspread_dataframe import set_with_dataframe

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=hxqGtrQBjBBTa8su14hRlxSnxxFFQ6&prompt=consent&access_type=offline&code_challenge=N-3kxMjNaTGEBjhNQ_uh5GZAepnRuxLnvv5XLySdEtA&code_challenge_method=S256

Enter verification code: 4/1AX4XfWilOf4LmkIb1Ss5SEJ687k4CqE-Vzo5bS2aUQFGny8Sj9VfpXOu7OM


## Manual Update

#### Query


In [33]:
!rm -r Job-listing-Query
!git clone https://github.com/alexisnpavlik/Job-listing-Query.git

main_query=open("""/content/Job-listing-Query/main_query.sql""", 'r').read()
organizations_query=open("""/content/Job-listing-Query/organizations_query.sql""", 'r').read()

Cloning into 'Job-listing-Query'...
remote: Enumerating objects: 234, done.
remote: Counting objects: 100% (234/234), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 234 (delta 141), reused 161 (delta 71), pack-reused 0
Receiving objects: 100% (234/234), 31.86 KiB | 741.00 KiB/s, done.
Resolving deltas: 100% (141/141), done.


In [35]:
main_df = pd.DataFrame(gallifrey.discovery.execute_query(main_query))
organizations_df = pd.DataFrame(gallifrey.solar.execute_query(organizations_query))

In [36]:
main_df.head(10)

,ID,Job title,Company_id,location,Created date,Approved date,Commited date,Account manager,Type of job,Type of service,Language of the post,Status,Last changes,Closing Date,Mutual Matches,Disqualified,Completed Applications,Incomplete applications,Others,Active,Pending for review,Hires,Sharing token
0,1782830,Desarrollador Flutter,730852,None,2022-01-24,2022-01-25,2022-01-26,Juan Felipe Bonilla García,full-time,prime,en,open,2022-02-01,2022-03-02,46.0,166.0,114.0,166.0,32.0,78.0,35.0,2022-01-28,FMZ2lb5M8xpY
1,1782796,Analista de Sistemas (Bogotá),332263,"Bogotá, Colombia",2022-01-24,2022-01-24,2022-01-24,Juan David Pinilla Boude,full-time,prime,en,closed,2022-01-29,2022-01-29,40.0,130.0,125.0,130.0,40.0,80.0,45.0,2022-01-25,58oGUmUh88Xs
2,1760981,Profesor de alemán,797801,None,2022-01-21,2022-01-21,2022-02-01,Juan David Pinilla Boude,full-time,prime,es,open,2022-01-29,2022-02-20,4.0,32.0,4.0,32.0,4.0,8.0,-4.0,2022-01-28,TTWkmqpui8p5
3,1714727,Sales Director,381186,None,2022-01-14,2022-01-14,2022-01-14,Juan David Pinilla Boude,part-time,prime,en,closed,2022-01-25,2022-01-25,15.0,159.0,132.0,159.0,6.0,21.0,110.0,2022-01-24,ni3qJ6oFFc0O
4,1704695,Recruiting Manager,785782,None,2022-01-13,2022-01-13,2022-01-14,Juan David Pinilla Boude,full-time,prime,es,closed,2022-01-27,2022-01-25,77.0,1463.0,854.0,1463.0,7.0,84.0,769.0,2022-01-27,cH9UXWSrHpi2
5,1704675,Ingeniero de Desarrollo .NetCore,740780,Colombia,2022-01-13,2022-01-14,2022-01-15,Juan David Pinilla Boude,full-time,prime,es,open,2022-01-31,2022-02-12,48.0,255.0,93.0,255.0,15.0,63.0,29.0,2022-01-27,1xQ19v3tdqDs
6,1697776,Inside Sales Agent / Appointment Setter,748285,None,2022-01-12,2022-01-13,2022-01-14,Melisa Medina Gomez,full-time,prime,es,closed,2022-01-31,2022-01-28,25.0,60.0,51.0,60.0,22.0,47.0,4.0,2022-01-31,gqnD2IwJmQLP
7,1689657,Client Success Coordinator,960840,None,2022-01-11,2022-01-12,2022-01-13,Juan Felipe Bonilla García,full-time,prime,en,closed,2022-01-27,2022-01-19,42.0,102.0,81.0,102.0,37.0,79.0,2.0,2022-01-26,O0TmBhG3lKtd
8,1689655,Desarrollador PHP,443430,None,2022-01-11,2022-01-11,2022-01-11,Juan Felipe Bonilla García,full-time,prime,es,closed,2022-01-28,2022-01-28,11.0,74.0,34.0,74.0,5.0,16.0,17.0,2022-01-28,cmz2NSxRt5PS
9,1689649,Tech leader,730852,Colombia,2022-01-11,2022-01-11,2022-01-11,Juan Felipe Bonilla García,full-time,prime,en,closed,2022-01-24,2022-01-24,9.0,60.0,19.0,60.0,6.0,15.0,4.0,2022-01-18,YqnMK9ZRfSJa


In [37]:
organizations_df.head(10)

,Company_id,Company
0,795908,
1,773085,0 2 Convergencia Digital S A S
2,763230,2 Way Sports
3,795148,23Ky Colombia S.A.S.
4,891553,60 Decibels
5,760653,Aberdare College
6,761403,"Absolute Computer Solutions, Inc."
7,805948,ABUSA Lab
8,786038,Academia de Inglés de la Universidad de Tarapacá
9,786905,Academia de Música DEUS


In [38]:
# Alfa ID Generator
def alfa_id(id):
  return hasher.encode(id)

# URL Generator
def url_generator(id):
  return 'https://torre.co/postings/'+hasher.encode(id)

# Manager URL Generator
def manager_generator(id):
  return 'https://torre.co/postings/'+hasher.encode(id)+'/members-list?sharingToken='

# Transforms columns
main_df.insert(1,"Alfa ID", main_df['ID'].apply(alfa_id),True)
main_df.insert( 2,"URL", main_df['ID'].apply(url_generator),True)
main_df.insert( 3,"Manager link", main_df['ID'].apply(manager_generator),True)
main_df['Manager link']=main_df['Manager link'].apply(lambda x:x+main_df['Sharing token'])

In [39]:
opportunities_df = pd.merge(left=main_df,right=organizations_df)

opportunities_df = opportunities_df.reindex(columns=['ID','Job title','Company','Alfa ID','location','Created date','Approved date','Commited date','Account manager','Type of job',
                                             'Type of service','URL','Manager link','Language of the post','Status','Mutual Matches','Disqualified','Last changes','Closing Date',
                                             'Completed Applications','Incomplete applications','Others','Active','Pending for review','Hires'])

In [ ]:
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1V1RkMTrOFEQLsYGJaA9KukO9fgtgf6uwG8o-QHrkk2Q/edit#gid=0').sheet1

sheet.clear()

set_with_dataframe(sheet, opportunities_df) 

# Update Google Sheet

In [40]:
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1V1RkMTrOFEQLsYGJaA9KukO9fgtgf6uwG8o-QHrkk2Q/edit#gid=0').sheet1

sheet.clear()

set_with_dataframe(sheet, opportunities_df) 

# Save to Google drive

##### Save csv

In [ ]:
drive.mount('/content/drive')

In [ ]:
opportunities_df.to_csv('/content/drive/My Drive/Data_studio_dataset/Dataset.csv',index=False,encoding='utf-8')

##### Save  Excel

In [ ]:
drive.mount('/content/drive')

In [ ]:
opportunities_df.to_excel('/drive/My Drive/Data_studio_dataset/Dataset.xls',index=False,encoding='utf-8')

# Testing

In [ ]:
query_testing = """

select id as 'Company_id', name as 'Company' from organizations ;

"""

In [ ]:
Testing = pd.DataFrame(gallifrey.solar.execute_query(query_testing))

### Update Google Sheet

In [ ]:
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1V1RkMTrOFEQLsYGJaA9KukO9fgtgf6uwG8o-QHrkk2Q/edit#gid=0').sheet1

sheet.clear()

set_with_dataframe(sheet, Testing) 

In [ ]:
Testing.head()

,Company_id,Company
0,795908,
1,773085,0 2 Convergencia Digital S A S
2,763230,2 Way Sports
3,795148,23Ky Colombia S.A.S.
4,891553,60 Decibels
